In [ ]:
from tensorflow.keras import layers, initializers, callbacks, models, optimizers, activations, backend, losses, metrics
from tensorflow.keras.wrappers import SKLearnRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

In [ ]:
X, y = make_regression(n_samples=100000,
                                     n_features=10,
                                     n_informative=4,
                                     noise=0.1,
                                     random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
K = backend

class ExponentialLearningRate(callbacks.Callback):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor
        self.rates = []
        self.losses = []

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        # get current LR
        lr = K.get_value(self.model.optimizer.learning_rate)
        self.rates.append(lr)
        self.losses.append(logs.get("loss"))
        # compute new LR
        new_lr = lr * self.factor
        # set new LR (works with modern Keras)
        self.model.optimizer.learning_rate.assign(new_lr)

In [ ]:
run_index = 1  # increment this at every run
run_logdir = os.path.join(os.curdir, "regressor_logs", "run_{:03d}".format(run_index))

tensorboard_cb = callbacks.TensorBoard(run_logdir)

In [ ]:
early_stop = callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

expon_lr = ExponentialLearningRate(factor=1.005)

In [ ]:
def build_model(X, y, n_hidden=1, n_neurons=30, learning_rate=0.0001):
    input_shape = X.shape[1:]
    model = models.Sequential()
    model.add(layers.InputLayer(shape=input_shape))
    for _ in range(n_hidden):
        model.add(layers.Dense(units=n_neurons,
                               activation=activations.relu,
                               kernel_initializer=initializers.HeNormal()))
    model.add(layers.Dense(units=1))
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=losses.mse,
                  optimizer=optimizer,
                  metrics=[metrics.mse, metrics.mae, metrics.mape, metrics.huber])
    return model

In [ ]:
estimator = SKLearnRegressor(model=build_model)

In [ ]:
history = estimator.fit(X_train,
                        y_train,
                        epochs=1,
                        validation_data=(X_valid, y_valid),
                        callbacks=[
                            early_stop,
                            expon_lr,

                        ],
                        verbose=2)

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d


idx = np.argmin(expon_lr.losses)
best_lr_sweep = expon_lr.rates[idx] / 10
print("LR-finder min loss lr:", best_lr_sweep)
print("loss at that lr:", expon_lr.losses[idx])
print("min/max lr in sweep:", min(expon_lr.rates), max(expon_lr.rates))

smoothed = gaussian_filter1d(expon_lr.losses, sigma=2)

plt.plot(expon_lr.rates, expon_lr.losses, alpha=0.3, label='batch loss')
plt.plot(expon_lr.rates, smoothed, label='smoothed loss')
plt.xscale('log')
plt.scatter([best_lr_sweep], [expon_lr.losses[idx]], color='red', label=f'best lr {best_lr_sweep:.2e}')
plt.legend()
plt.xlabel("Learning rate")
plt.ylabel("Loss")
plt.show()

In [ ]:
param_grid = {
    "model_kwargs": [
        {"n_hidden": h, "n_neurons": n, "learning_rate": lr}
        for h in [1, 2, 3, 4]
        for n in [16, 32, 64, 128]
        for lr in [0.03, 0.003, 0.0003]
    ]
}

search = RandomizedSearchCV(estimator,
                            param_distributions=param_grid,
                            n_iter=10,
                            cv=3,
                            scoring=""
                                    "",
                            verbose=2)

search.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[early_stop], verbose=0)

In [ ]:
best_model = search.best_estimator_
best_params = search.best_params_
print("Best parameters:", best_params)

In [ ]:
best_params = {'model_kwargs': {'n_hidden': 2, 'n_neurons': 128, 'learning_rate': 0.044}}

In [ ]:
y_pred= best_model.predict(X_valid)
mse = ((y_valid - y_pred.flatten()) ** 2).mean()
print("Best model MSE on validation set:", mse)

In [ ]:
estimator = SKLearnRegressor(model=build_model, **best_params)

In [ ]:
history = estimator.fit(X_train,
                        y_train,
                        epochs=100,
                        validation_data=(X_valid, y_valid),
                        callbacks=[
                            early_stop,
                            tensorboard_cb
                        ],
                        verbose=2)